In [1]:
import pandas as pd

In [8]:
!pip install -q tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00


# Import Data / Transform into Dataframes

In [2]:
# Needed you you are running in colab and want to read from google drive.

from google.colab import drive
drive.mount("/content/drive")

%cd '/content/drive/MyDrive/gatech_coursework/deep_learning_final_project/recommender-system-main'

Mounted at /content/drive
/content/drive/MyDrive/gatech_coursework/deep_learning_final_project/recommender-system-main


In [27]:
# Data pre processing.

df_loaded = pd.read_csv('data/processed_data.dat')
df_loaded.head()

df_loaded.columns

df_short = df_loaded[['rating', 'occupation', 'age_group_child', 'age_group_teen', 'age_group_adult', 'age_group_senior', 'gender_F', 'gender_M', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

df_short.head()

# distinct_elements = pd.unique(df_short.values.ravel())
# print(distinct_elements)


# df_sampled = df_short.sample(n=100000, random_state=42).astype(np.float32)
# y = df_sampled[['rating']].values.flatten()
# np.unique(y, return_counts=True)

(array([0., 1.], dtype=float32), array([22217, 77783]))

In [25]:

import tensorflow as tf
from tensorflow.keras import layers, Model
import tensorflow_recommenders as tfrs
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Simulate some input data
def GetData(n_samples, input_dim):
    df_sampled = df_short.sample(n=n_samples, random_state=42).astype(np.float32)
    X = df_sampled.loc[:, df_sampled.columns != 'rating'].values
    y = df_sampled[['rating']].values.flatten()
    return X, y


# DCN-V2 Full Model
class DCNV2Model(Model):
    def __init__(self, input_dim, projection_dim=64, num_cross_layers=3, dnn_units=[256, 128]):
        super(DCNV2Model, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, projection_dim)
        self.cross_layers = tf.keras.Sequential()
        for _ in range(num_cross_layers):
          self.cross_layers.add(tfrs.layers.dcn.Cross(projection_dim=projection_dim, kernel_initializer="glorot_uniform"))
        self.dnn = tf.keras.Sequential()
        for units in dnn_units:
            self.dnn.add(layers.Dense(units, activation='relu'))
        self.final_dense = layers.Dense(1, activation='sigmoid')  # For binary classification

    def call(self, inputs):
        print("inputs.shape", inputs.shape)
        embed = self.embedding_layer(inputs)
        cross_out = self.cross_layers(embed)
        dnn_out = self.dnn(inputs)
        output = self.final_dense(dnn_out)
        return output

# Get data
# input_dim = 32
input_dim = 25
X, y = GetData(n_samples=100000, input_dim=input_dim)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create the model
model = DCNV2Model(input_dim=input_dim, projection_dim=16, num_cross_layers=3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'Accuracy'])

# Train the model
model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
print("Test ROC AUC:", roc_auc_score(y_test, y_pred))


Epoch 1/10
inputs.shape (None, 25)
inputs.shape (None, 25)


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['dcnv2_model_18/embedding_14/embeddings', 'dcnv2_model_18/sequential_14/cross_33/dense_66/kernel', 'dcnv2_model_18/sequential_14/cross_33/dense_67/kernel', 'dcnv2_model_18/sequential_14/cross_33/dense_67/bias', 'dcnv2_model_18/sequential_14/cross_34/dense_68/kernel', 'dcnv2_model_18/sequential_14/cross_34/dense_69/kernel', 'dcnv2_model_18/sequential_14/cross_34/dense_69/bias', 'dcnv2_model_18/sequential_14/cross_35/dense_70/kernel', 'dcnv2_model_18/sequential_14/cross_35/dense_71/kernel', 'dcnv2_model_18/sequential_14/cross_35/dense_71/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


inputs.shape (None, 25)
560/563 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6010 - Accuracy: 0.7775 - loss: 0.5212inputs.shape (None, 25)
563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - AUC: 0.6011 - Accuracy: 0.7775 - loss: 0.5212 - val_AUC: 0.6451 - val_Accuracy: 0.7769 - val_loss: 0.5136
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - AUC: 0.6394 - Accuracy: 0.7797 - loss: 0.5067 - val_AUC: 0.6550 - val_Accuracy: 0.7759 - val_loss: 0.5063
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - AUC: 0.6490 - Accuracy: 0.7786 - loss: 0.5058 - val_AUC: 0.6662 - val_Accuracy: 0.7766 - val_loss: 0.5040
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - AUC: 0.6619 - Accuracy: 0.7798 - loss: 0.5001 - val_AUC: 0.6659 - val_Accuracy: 0.7776 - val_loss: 0.5048
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - AUC: 0.6662 - Accuracy: 0.7827 - loss: 0.4942 - val_AUC: 0.6668 - val_Accuracy: 0.7780 - val_loss: 0.5017
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - AUC: 0.6658 - Accurac